# Notebook to calculate the rotation of the MIRI images with respect to NIRCam
Let's start with the imports

In [2]:
from astropy.io import fits
from astropy.wcs import WCS
import math
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from scipy.ndimage import rotate
from reproject import reproject_interp


Define functions:

In [30]:

def rotate_image(image, angle):
    """Rotate the image by the given angle"""
    return rotate(image, angle, reshape=False, order=1, mode='nearest')  # Rotate by negative angle

def process_fits(fits_file, output_folder):
    """Process the MIRI FITS file to rotate and save a new aligned FITS file."""
    
    output_filename = os.path.join(output_folder, 'rotated_'+os.path.basename(fits_file))
    
    with fits.open(fits_file) as hdul:
        image_data = hdul[1].data
        header = hdul[1].header
        wcs = WCS(header)

        # Get original reference pixel (before rotation)
        crpix_original = np.array([header['CRPIX1'], header['CRPIX2']])
        
        print("CRPIX1:", header['CRPIX1'], "CRPIX2:", header['CRPIX2'])
        print("Image shape:", image_data.shape)

        print(crpix_original)
        # Get world coordinates of the original reference pixel
        crval_original = wcs.pixel_to_world(crpix_original[0], crpix_original[1])

        # Extract rotation angle from PC matrix
        if 'PC1_1' in header and 'PC2_2' in header:
            pc11 = header['PC1_1'] 
            pc12 = header['PC1_2'] 
            pc21 = header['PC2_1'] 
            pc22 = header['PC2_2'] 
            pc_matrix = np.array([[pc11, pc12], 
                                    [pc21, pc22]])
            angle = 180 - np.arccos(pc11) * 180 / np.pi
            print(f"Rotating by {angle:.2f} degrees...")        
        else:
            print("No PC matrix found, assuming no rotation")
            angle = 0

        # Update the WCS: Rotate PC matrix by -angle (counterclockwise to match sky)
        theta = np.radians(-angle)  # Convert to radians (negative for counterclockwise)
        cos_t, sin_t = np.cos(theta), np.sin(theta)

        #pc11_new = pc11*cos_t - pc21*sin_t
        #pc12_new = pc12*cos_t - pc22*sin_t
        #pc21_new = pc11*sin_t + pc21*cos_t
        #pc22_new = pc12*sin_t + pc22*cos_t

        #header['PC1_1'], header['PC1_2'] = pc11_new, pc12_new
        #header['PC2_1'], header['PC2_2'] = pc21_new, pc22_new
        

        # Apply rotation to PC matrix
        rotation_matrix = np.array([[cos_t, -sin_t], [sin_t, cos_t]])
        new_pc_matrix = np.dot(rotation_matrix, pc_matrix)
        
        header['PC1_1'], header['PC1_2'] = new_pc_matrix[0]
        header['PC2_1'], header['PC2_2'] = new_pc_matrix[1]
        
        # Create new WCS with updated PC matrix
        new_wcs = WCS(header)

        # Convert original world coordinates back to pixel in the new WCS
        new_crpix = new_wcs.world_to_pixel(crval_original)

        # Update reference pixel to keep sky alignment fixed
        header['CRPIX1'], header['CRPIX2'] = new_crpix

        # Rotate image
        rotated_image = rotate_image(image_data, angle)

    # Ensure output directory exists
    os.makedirs(output_folder, exist_ok=True)

    # Create new FITS file with rotated image
    hdu = fits.PrimaryHDU(rotated_image, header=header)
    hdul_rotated = fits.HDUList([hdu])
    hdul_rotated.writeto(output_filename, overwrite=True)
    
    print(f"Rotated image saved as: {output_filename}")

    # Plot original and rotated images
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    axs[0].imshow(image_data, cmap='gray', origin='lower')
    axs[0].set_title('Original Image')
    axs[1].imshow(rotated_image, cmap='gray', origin='lower')
    axs[1].set_title('Rotated Image (North Up, East Left)')
    plt.show()


Read in the data files and define output directory

In [4]:

# Load datasets
primer = '/home/bpc/University/master/Red_Cardinal/MIRI/PRIMER/'
primer_data = glob.glob(os.path.join(primer, "*.fits"))
print(f"Found {len(primer_data)} FITS files.")

cweb = "/home/bpc/University/master/Red_Cardinal/MIRI/COSMOS-Web/"
cweb_data = glob.glob(os.path.join(cweb, "*.fits"))
print(f"Found {len(cweb_data)} FITS files.")

# Create output directory
output_dir = "./rotated/"
os.makedirs(output_dir, exist_ok=True)

#cutout_name =  f"./cutouts/7102_F770W_cutout_primer.fits"
cutout_name =  f"./cutouts/10592_F770W_cutout_primer.fits"
patch = f"./MIRI/PRIMER/jw01837-o003_t003_miri_f770w_i2d.fits"

Found 4 FITS files.
Found 164 FITS files.


Now we call the rotation function

In [31]:
process_fits(cutout_name, output_dir)


CRPIX1: 667.29944875089 CRPIX2: -122.58176297208
Image shape: (48, 48)
[ 667.29944875 -122.58176297]
Rotating by 63.17 degrees...


ValueError: Illegal value: array(667.64316606).

In [16]:
cutout_name =  f"./cutouts/10592_F770W_cutout_primer.fits"

with fits.open(cutout_name) as hdul:
    data = hdul[1].data
    header = hdul[1].header
    wcs = WCS(header)

# Extract rotation angle from PC matrix
if 'PC1_1' in header and 'PC2_2' in header:
    pc_matrix = np.array([[header['PC1_1'], header['PC1_2']], 
                            [header['PC2_1'], header['PC2_2']]])
    angle = np.arccos(-1*pc_matrix[0, 0]) * 180 / np.pi
    print("First entry:", pc_matrix[0,0])
    print(header['PC1_1'])
    print(header['PC1_2'])
    print(header['PC2_1'])
    print(header['PC2_2'])
    print("Angle: ", angle)
    print(pc_matrix)
else:
    print("No PC matrix found, assuming no rotation")
    angle = 0

First entry: -0.45139107874244
-0.45139107874244
-0.89232622623777
-0.89232622623777
0.45139107874244
Angle:  63.167030798708026
[[-0.45139108 -0.89232623]
 [-0.89232623  0.45139108]]


Set OBSGEO-B to     8.121531 from OBSGEO-[XYZ].
Set OBSGEO-H to 1684160522.172 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
